## 0. Setup

In [ ]:
# !pip install PyPDF2 langchain tiktoken chromadb openai pytest pytest-mock pydantic

## 1. Extracting files

In [ ]:
# pdf_loader.py
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extrae todo el texto de un archivo PDF.
    Args:
      pdf_path (str): Ruta al archivo PDF.
    Returns:
      str: Texto completo extraído.
    """
    reader = PdfReader(pdf_path)
    full_text = []
    for page in reader.pages:
        full_text.append(page.extract_text())
    return "\n".join(filter(None, full_text))

# Test de unidad básico
if __name__ == "__main__":
    pdf_path = "./test_docs/manual_empleado.pdf"
    text = extract_text_from_pdf(pdf_path)
    print(f"Extracted text length: {len(text)}")

## 2. Text Splitting into Chunks

In [ ]:
# chunking.py
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List

def chunk_text(text: str, chunk_size: int = 1000, chunk_overlap: int = 100) -> List[str]:
    """
    Divide el texto en chunks compatibles con LangChain.
    
    Args:
      text (str): Texto completo a dividir.
      chunk_size (int): Tamaño máximo por chunk.
      chunk_overlap (int): Tamaño de overlap para mantener contexto.
      
    Returns:
      List[str]: Lista de fragmentos de texto.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

# Test unidad básico
if __name__ == "__main__":
    chunks = chunk_text(text)
    print(f"Chunks created: {len(chunks)}")
    print(f"First chunk preview: {chunks[0][:2000]}")

## 3. Embedding

In [ ]:
# embeddings.py
import os
from langchain.embeddings import OpenAIEmbeddings
import tiktoken
from torch import chunk

def batch_texts_by_token_limit(texts: list[str], max_tokens: int = 300000, model_name: str = "text-embedding-3-small") -> list[list[str]]:
    """
    Divide la lista de textos en lotes para no exceder el límite de tokens por solicitud.
    """
    enc = tiktoken.encoding_for_model(model_name)
    batches = []
    current_batch = []
    current_tokens = 0
    for text in texts:
        tokens = len(enc.encode(text))
        if current_tokens + tokens > max_tokens and current_batch:
            batches.append(current_batch)
            current_batch = []
            current_tokens = 0
        current_batch.append(text)
        current_tokens += tokens
    if current_batch:
        batches.append(current_batch)
    return batches

def create_openai_embeddings(texts: list[str]) -> list[list[float]]:
    """
    Crea embeddings usando el modelo text-embedding-3-small de OpenAI, respetando el límite de tokens por solicitud.
    Args:
      texts (list[str]): Lista de textos a vectorizar.
      openai_api_key (str): API Key OpenAI.
    Returns:
      list[list[float]]: Vectores embedding.
    """
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
    batches = batch_texts_by_token_limit(texts)
    vectors = []
    for batch in batches:
        vectors.extend(embedding_model.embed_documents(batch))
    return vectors

# Test básico
if __name__ == "__main__":
    embeddings = create_openai_embeddings(chunks)
    print(f"Embedding vector length for first text: {len(embeddings[0])}")

## 4. Vector Stores

In [ ]:
# vector_store.py
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import os

def save_embeddings_in_chromadb(
    texts: list[str],
    metadatas: list[dict],
    persist_directory: str
):
    """
    Guarda fragmentos y embeddings en ChromaDB para búsqueda.
    Args:
      texts: Lista de documentos/textos.
      metadatas: Lista de diccionarios con metadatos, mismo orden que texts.
      persist_directory (str): Carpeta donde persistir la DB.
      openai_api_key (str): API Key OpenAI para crear embeddings.
    """
    os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
    
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = Chroma(
        collection_name="internal_docs",
        embedding_function=embedding_model,
        persist_directory=persist_directory
    )
    vectorstore.add_texts(texts=texts, metadatas=metadatas)
    vectorstore.persist()
    vectorstore = None  # Liberar memoria

    # Test básico
if __name__ == "__main__":
    sample_metadata = [{"source": pdf_path}]

    batches = batch_texts_by_token_limit(chunks)
    for batch in batches:
        save_embeddings_in_chromadb(batch, sample_metadata, "./db")
    print("Embeddings guardados con éxito.")


## 5. Retriving from the Persistant Vector Datastore

In [ ]:
# retriever.py
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from chromadb.config import Settings
import chromadb

def load_retriever(persist_directory: str, openai_api_key: str):
    """
    Carga la base ChromaDB como vectorstore para realizar consultas.
    Retorna el objeto retriever.
    """
    os.environ["OPENAI_API_KEY"] = openai_api_key
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = Chroma(
        collection_name="internal_docs",
        embedding_function=embedding_model,
        persist_directory=persist_directory
    )
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
    return retriever

# Test básico: recuperar textos similares
if __name__ == "__main__":
    retriever = load_retriever("./db", os.getenv("OPENAI_API_KEY"))
    query = "¿Cuáles son las políticas de seguridad?"
    docs = retriever.get_relevant_documents(query)
    print(f"Documentos recuperados: {len(docs)}")
    print(f"Primera doc preview: {docs[0].page_content[:300] if docs else 'Sin resultados'}")

## 6. Retrivers in Langchain

In [ ]:
# rag_answer.py
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os

def generate_answer_from_rag(question: str, retriever, openai_api_key: str) -> str:
    """
    Dada una pregunta y un retriever, genera una respuesta usando RAG y modelo GPT.
    
    Args:
      question (str): Pregunta a responder.
      retriever: Retriever previamente inicializado (ChromaDB).
      openai_api_key (str): API key para OpenAI.
      
    Returns:
      str: Respuesta generada.
    """
    os.environ["OPENAI_API_KEY"] = openai_api_key
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    result = qa_chain.run(question)
    return result

# Test básico
if __name__ == "__main__":
    openai_api_key = os.getenv("OPENAI_API_KEY")
    retriever = load_retriever("./db", openai_api_key)
    question = "¿Qué procedimientos debo seguir en caso de emergencia laboral?"
    answer = generate_answer_from_rag(question, retriever, openai_api_key)
    print(f"Respuesta:\n{answer}")

In [ ]:
# test_rag_system.py
import pytest
import os
import shutil

@pytest.fixture(scope="module")
def setup_teardown():
    """
    Fixture para preparar entorno con embedding y textos de ejemplo y limpiar después.
    """
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if openai_api_key is None:
        pytest.skip("OPENAI_API_KEY no está configurado en el entorno de prueba.")
    
    # Paso 1: Extraer texto dummy (se podría usar pdf real aquí)
    sample_text = "Las políticas de seguridad establecen que " \
                  "todo empleado debe reportar incidentes.\n" \
                  "En caso de emergencia laboral, sigue el protocolo ABC.\n" \
                  "El horario de trabajo es de 9 a 18 hrs con pausa para almuerzo.\n" \
                  "Los reportes trimestrales están basados en indicadores XYZ.\n" \
                  "Cualquier cambio en política debe ser aprobado por RRHH."
    chunks = chunk_text(sample_text)
    metadatas = [{"source": f"chunk-{i}"} for i in range(len(chunks))]
    
    persist_dir = "./test_db"
    
    # Guardar embeddings
    save_embeddings_in_chromadb(chunks, metadatas, persist_dir, openai_api_key)
    
    yield openai_api_key, persist_dir
    
    # Cleanup
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)


@pytest.mark.parametrize("question", [
    "¿Cuáles son las políticas de seguridad?",
    "¿Qué hacer en caso de emergencia laboral?",
    "¿Cuál es el horario de trabajo?",
    "¿En qué se basan los reportes trimestrales?",
    "¿Quién aprueba los cambios en las políticas?"
])
def test_rag_answers(setup_teardown, question):
    openai_api_key, persist_dir = setup_teardown
    retriever = load_retriever(persist_dir, openai_api_key)
    answer = generate_answer_from_rag(question, retriever, openai_api_key)
    
    assert isinstance(answer, str)
    assert len(answer.strip()) > 0
    # Extra simple: la respuesta debe tener al menos 10 caracteres.
    assert len(answer) > 10


# Seguridad básica: test contra inyección SQL simulada (en este contexto vectors, es más prevención de inputs malignos)
def test_sql_injection_prevention(setup_teardown):
    openai_api_key, persist_dir = setup_teardown
    retriever = load_retriever(persist_dir, openai_api_key)
    injection_string = "'; DROP TABLE users; --"
    answer = generate_answer_from_rag(injection_string, retriever, openai_api_key)
    # Validamos que el código no explota y responde algo
    assert answer is not None


# Contract test: Validar esquema Response OpenAPI simple (mock de esquema)
from pydantic import BaseModel, ValidationError

class OpenAPIResponseSchema(BaseModel):
    answer: str

def test_openapi_contract(setup_teardown):
    openai_api_key, persist_dir = setup_teardown
    retriever = load_retriever(persist_dir, openai_api_key)
    question = "¿Cuál es el protocolo de emergencias?"
    answer = generate_answer_from_rag(question, retriever, openai_api_key)
    try:
        # Simular que la respuesta viene en formato JSON { "answer": <respuesta> }
        data = {"answer": answer}
        validated = OpenAPIResponseSchema(**data)
        assert validated.answer == answer
    except ValidationError:
        pytest.fail("Respuesta no cumple esquema OpenAPI esperado")